In [1]:
using Agents
#using Random: Xoshiro # access the RNG object

In [2]:
using Statistics: mean

In [16]:
#using CairoMakie
#using InteractiveDynamics

In [3]:
#模型中不变参数设置
L = 3 #初始矩阵大小，对应L*L矩阵


2

In [3]:
#模型中可变参数设置
betray=2  #背叛数量>=2 压力+1
temp = 1.2  #诱惑值
#payMtx=[1 0; b 0]
alpha =  0.3 
theta = 20  #压力缺口上限
Kmax = 1 
lamda = 10 #额外阈值


10

In [4]:
size = (L, L)

(3, 3)

In [11]:
space = GridSpaceSingle(size; periodic = true, metric = :manhattan)

GridSpaceSingle with size (3, 3), metric=manhattan, periodic=true

In [6]:
@agent struct SchellingAgent(GridAgent{2}) 
    #mood::Bool # whether the agent is happy in its position
    #group::Int # The group of the agent, determines mood as it interacts with neighbors
    A_self::Int   #伦理
    A_peer::Int   #收益
    #A_mtx::Float64  #压力
    revenue::Float64  #收益
    #revenue_nb_av::Float64  #邻居的平均收益
    #P1::Float64   #第一层概率
    #P2::Float64   #第二层概率
    #Pmix::Float64  #概率相乘
    strate::Int   #1-合作 0-背叛
    overstress::Int  #过度压力
end

In [7]:
properties = Dict(:min_betray => betray, :temp => temp,
:alpha => alpha, :theta => theta, :lamda => lamda, :Kmax => Kmax)

Dict{Symbol, Any} with 6 entries:
  :alpha      => 0.3
  :min_betray => 2
  :Kmax       => 1
  :payMtx     => [1.0 0.0; 1.2 0.0]
  :lamda      => 10
  :theta      => 20

In [12]:
function schelling_step_1!(agent, model)
    # Here we access a model-level property `min_to_be_happy`.
    # This will have an assigned value once we create the model.
    A = [1 0; model.temp 0] 
    #以下是针对当前单个代理的指令
    payoff =0
    a=[agent.strate, 1-agent.strate]
    PA=transpose(a) * A
    # For each neighbor, get group and compare to current agent's group
    # and increment `count_neighbors_same_group` as appropriately.
    # Here `nearby_agents` (with default arguments) will provide an iterator
    # over the nearby agents one grid cell away, which are at most 8.
    for neighbor in nearby_agents(agent, model)
        b=[neighbor.strate, 1-neighbor.strate]
        pay0 = PA * b
        payoff += PA * b
    end
    agent.revenue=payoff
    return
end

schelling_step_1! (generic function with 1 method)

In [13]:
function schelling_step_2!(agent, model)
    # Here we access a model-level property `min_to_be_happy`.
    # This will have an assigned value once we create the model.
    minbetray = model.min_betray
    alpha = model.alpha
    theta = model.theta
    lamda = model.lamda
    Kmax = model.Kmax
    #以下是针对当前单个代理的指令
    count_betray = 0
    revenue_nb = 0
    # For each neighbor, get group and compare to current agent's group
    # and increment `count_neighbors_same_group` as appropriately.
    # Here `nearby_agents` (with default arguments) will provide an iterator
    # over the nearby agents one grid cell away, which are at most 8.
    for neighbor in nearby_agents(agent, model)
        if neighbor.strate==0
            count_betray += 1
        end
        revenue_nb += neighbor.revenue
    end
    #nearby_number = length(nearby_agents(agent, model))
    nearby_number = 4
    revenue_nb_av=revenue_nb/nearby_number
    bet2= (count_betray>=2)
    pe2=(agent.revenue <= revenue_nb_av)
    #更新压力矩阵A
    agent.A_self += (count_betray>=2)
    agent.A_peer += (agent.revenue <= revenue_nb_av) #计算朋辈压力
    A_mtx = (1-alpha) * agent.A_self + alpha * agent.A_peer #更新压力矩阵A
    #迭代&策略更新
    A_gap=maximum([theta-A_mtx,0])
    P1 = exp(-A_gap)  #计算概率P1
    ragent = random_agent(model)
    K_A = Kmax*A_gap/theta 
    drevenue= agent.revenue -ragent.revenue
    P2 = 1 / (1+ exp(drevenue/K_A))
    isnan(P2) && (P2= 0) #处理NaN值赋0  #0/0先赋值为0，然后赋值为0.5
    Pmix=P1*P2
    if Pmix > rand() 
        agent.strate = ragent.strate
        agent.A_peer = 0 #清空改变策略的Ap
        agent.A_self = 0 #清空改变策略的As
    end
    #Fc(t+1) = sum(sum(Strmtx))/L^2  #在最后提取数据时计算
    agent.overstress = (A_mtx >= (theta+lamda))
    return
end

schelling_step_2! (generic function with 1 method)

In [14]:
function complex_model_step!(model)
    # tip: these schedulers should be defined as properties of the model
    for agent in allagents(model)
        schelling_step_1!(agent, model)
        schelling_step_2!(agent, model)
    end
    return
end

complex_model_step! (generic function with 1 method)

In [11]:
schelling = StandardABM(
    # input arguments
    SchellingAgent, space;
    # keyword arguments
    properties, # in Julia if the input variable and keyword are named the same,
                # you don't need to repeat the keyword!
    model_step! = complex_model_step!
)

StandardABM with 0 agents of type SchellingAgent
 agents container: Dict
 space: GridSpaceSingle with size (3, 3), metric=manhattan, periodic=true
 scheduler: fastest
 properties: alpha, min_betray, Kmax, payMtx, lamda, theta

In [ ]:
function wealth_model_2D(; dims = (25, 25), wealth = 1, M = 1000)  #1000个代理
    space = GridSpace(dims, periodic = true)  #定义 空间  #此处一个单元可以由多个代理 GridSpaceSingle - 单个代理
    model = ABM(WealthInSpace, space; agent_step! = agent_step_2d!,
                scheduler = Schedulers.Randomly())  #定义模型，包含 空间 代理 进化规则 属性 调度程序（调度者）
    for _ in 1:M # add agents in random positions
        add_agent!(model, wealth)
    end
    return model  
end

In [12]:
for n in 1:L^2
    add_agent_single!(schelling; revenue=0, A_self=0, A_peer=0, strate = rand(0:1), overstress=0)
end

In [19]:
schelling[1].id

1

In [20]:
for n in 1:L^2
    aa=schelling[n].id
    println("agent=$aa")
end

agent=1
agent=2
agent=3
agent=4
agent=5
agent=6
agent=7
agent=8
agent=9


In [22]:
for n in 1:L^2
    println("agent=$schelling[n]")
end

agent=StandardABM with 9 agents of type SchellingAgent
 agents container: Dict
 space: GridSpaceSingle with size (3, 3), metric=manhattan, periodic=true
 scheduler: fastest
 properties: alpha, min_betray, Kmax, payMtx, lamda, theta[n]
agent=StandardABM with 9 agents of type SchellingAgent
 agents container: Dict
 space: GridSpaceSingle with size (3, 3), metric=manhattan, periodic=true
 scheduler: fastest
 properties: alpha, min_betray, Kmax, payMtx, lamda, theta[n]
agent=StandardABM with 9 agents of type SchellingAgent
 agents container: Dict
 space: GridSpaceSingle with size (3, 3), metric=manhattan, periodic=true
 scheduler: fastest
 properties: alpha, min_betray, Kmax, payMtx, lamda, theta[n]
agent=StandardABM with 9 agents of type SchellingAgent
 agents container: Dict
 space: GridSpaceSingle with size (3, 3), metric=manhattan, periodic=true
 scheduler: fastest
 properties: alpha, min_betray, Kmax, payMtx, lamda, theta[n]
agent=StandardABM with 9 agents of type SchellingAgent
 agen

In [23]:
for n in 1:L^2
    println("agent=$(schelling[n])")
end

agent=SchellingAgent(1, (3, 3), 2, 2, 1.0, 1, 0)
agent=SchellingAgent(2, (3, 1), 0, 0, 4.8, 0, 0)
agent=SchellingAgent(3, (2, 3), 0, 0, 3.5999999999999996, 0, 0)
agent=SchellingAgent(4, (2, 2), 0, 0, 3.0, 1, 0)
agent=SchellingAgent(5, (2, 1), 2, 1, 2.0, 1, 0)
agent=SchellingAgent(6, (3, 2), 0, 0, 3.0, 1, 0)
agent=SchellingAgent(7, (1, 1), 2, 1, 2.0, 1, 0)
agent=SchellingAgent(8, (1, 3), 0, 0, 3.5999999999999996, 0, 0)
agent=SchellingAgent(9, (1, 2), 0, 0, 3.0, 1, 0)


In [21]:
for n in 1:L^2
    aa=schelling[n]
    println("agent=$aa")
end

agent=SchellingAgent(1, (3, 3), 2, 2, 1.0, 1, 0)
agent=SchellingAgent(2, (3, 1), 0, 0, 4.8, 0, 0)
agent=SchellingAgent(3, (2, 3), 0, 0, 3.5999999999999996, 0, 0)
agent=SchellingAgent(4, (2, 2), 0, 0, 3.0, 1, 0)
agent=SchellingAgent(5, (2, 1), 2, 1, 2.0, 1, 0)
agent=SchellingAgent(6, (3, 2), 0, 0, 3.0, 1, 0)
agent=SchellingAgent(7, (1, 1), 2, 1, 2.0, 1, 0)
agent=SchellingAgent(8, (1, 3), 0, 0, 3.5999999999999996, 0, 0)
agent=SchellingAgent(9, (1, 2), 0, 0, 3.0, 1, 0)


In [24]:
abmtime(schelling)

2

In [14]:
#step!(schelling, 3)

StandardABM with 16 agents of type SchellingAgent
 agents container: Dict
 space: GridSpaceSingle with size (4, 4), metric=manhattan, periodic=true
 scheduler: fastest
 properties: alpha, min_betray, Kmax, payMtx, lamda, theta

In [14]:
N = 2

2

In [19]:
#adata = [(:strate, mean), (:overstress, mean)]

2-element Vector{Tuple{Symbol, typeof(mean)}}:
 (:strate, Statistics.mean)
 (:overstress, Statistics.mean)

In [13]:
adata = [:strate, :revenue, :A_peer, :A_self, :overstress]

5-element Vector{Symbol}:
 :strate
 :revenue
 :A_peer
 :A_self
 :overstress

In [15]:
data, _ = run!(schelling, N; adata)
#data[(end-20):end, :]

id=SchellingAgent(5, (2, 1), 0, 0, 0.0, 1, 0).id, strate=SchellingAgent(5, (2, 1), 0, 0, 0.0, 1, 0).strate, position=SchellingAgent(5, (2, 1), 0, 0, 0.0, 1, 0).pos
nid=SchellingAgent(3, (2, 3), 0, 0, 0.0, 0, 0).id, nstrate=SchellingAgent(3, (2, 3), 0, 0, 0.0, 0, 0).strate, nposition=SchellingAgent(3, (2, 3), 0, 0, 0.0, 0, 0).pos
pay0=0.0
nid=SchellingAgent(7, (1, 1), 0, 0, 0.0, 1, 0).id, nstrate=SchellingAgent(7, (1, 1), 0, 0, 0.0, 1, 0).strate, nposition=SchellingAgent(7, (1, 1), 0, 0, 0.0, 1, 0).pos
pay0=1.0
nid=SchellingAgent(2, (3, 1), 0, 0, 0.0, 0, 0).id, nstrate=SchellingAgent(2, (3, 1), 0, 0, 0.0, 0, 0).strate, nposition=SchellingAgent(2, (3, 1), 0, 0, 0.0, 0, 0).pos
pay0=0.0
nid=SchellingAgent(4, (2, 2), 0, 0, 0.0, 1, 0).id, nstrate=SchellingAgent(4, (2, 2), 0, 0, 0.0, 1, 0).strate, nposition=SchellingAgent(4, (2, 2), 0, 0, 0.0, 1, 0).pos
pay0=1.0
payoff=2.0
id2=SchellingAgent(5, (2, 1), 0, 0, 2.0, 1, 0).id, strate2=SchellingAgent(5, (2, 1), 0, 0, 2.0, 1, 0).strate, position2=S

(27×7 DataFrame
 Row │ time   id     strate  revenue  A_peer  A_self  overstress 
     │ Int64  Int64  Int64   Float64  Int64   Int64   Int64      
─────┼───────────────────────────────────────────────────────────
   1 │     0      1       1      0.0       0       0           0
   2 │     0      2       0      0.0       0       0           0
   3 │     0      3       0      0.0       0       0           0
   4 │     0      4       1      0.0       0       0           0
   5 │     0      5       1      0.0       0       0           0
   6 │     0      6       1      0.0       0       0           0
   7 │     0      7       1      0.0       0       0           0
   8 │     0      8       0      0.0       0       0           0
  ⋮  │   ⋮      ⋮      ⋮        ⋮       ⋮       ⋮         ⋮
  21 │     2      3       0      3.6       0       0           0
  22 │     2      4       1      3.0       0       0           0
  23 │     2      5       1      2.0       1       2           0
  24 │     

In [18]:
schelling[1].pos

(2, 3)

In [21]:
schelling[2].pos

(1, 3)

In [22]:
schelling[3].pos

(2, 4)

In [23]:
id_in_position((1,1), schelling)

12